<a href="https://colab.research.google.com/github/valgh/colab_notebooks/blob/main/preprocessing_randwalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# now we can start working with our dataset.
# mounting drive...
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
# now we can access the drive
# and we retrieve the data
import os
data_path = "/content/gdrive/My Drive/data_4/"
list_dir = os.listdir(data_path)
print(list_dir)

['ags_polling_thread_run.json', 'ags_destroy_worker_init.json', 'ags_thread_test_sync_flags.json', 'ags_single_thread_run.json', 'ags_thread_first.json', 'ags_returnable_thread_new.json', 'ags_timestamp_set_flags.json', 'ags_message_delivery_find_sender.json', 'ags_task_class_init.json', 'ags_thread_get_property.json', 'ags_file_write_thread_list.json', 'ags_message_envelope_alloc.json', 'ags_polling_thread_connectable_interface_init.json', 'ags_destroy_worker_get_instance.json', 'ags_thread_list_resource.json', 'ags_thread_resume.json', 'ags_mutex_manager_remove.json', 'ags_condition_manager_insert.json', 'ags_thread_timer.json', 'ags_thread_application_context_set_value_callback.json', 'ags_returnable_thread_stop.json', 'ags_task_thread_run.json', 'ags_task_completion_list_resource.json', 'ags_thread_pool_init.json', 'ags_polling_thread_stop.json', 'ags_generic_main_loop_class_init.json', 'ags_mutex_manager_get_instance.json', 'ags_timestamp_set_unix_time.json', 'ags_history_class_in

In [ ]:
# we now make a dictionary out of the data directory: FUNC_NAME -> FUNC_JSON_DATA
import json
func_dict = {}
i=0
for func in list_dir:
  func_name = func[:-5]
  i+=1
  if i == 1000:
    print("reporting...\n")
    i=0
  with open(data_path+func) as f:
    func_json_data = json.load(f)
    func_dict[func_name] = func_json_data

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...

reporting...



In [ ]:
# now we save this to a JSON file.
import json
json.dump(func_dict, open("/content/gdrive/My Drive/func_dict_raw.json", "w"))

In [ ]:
# we can now access func_dict through the JSON file we saved
import json
func_dict = json.load(open("/content/gdrive/My Drive/func_dict_raw.json", "r"))

In [ ]:
import networkx as nx
from networkx.readwrite import json_graph

In [ ]:
# define functions to perform a random walk
# on each CFG
import numpy as np
import random

def get_entrypoint(cfg):
  entrypoint=None
  attr = nx.get_node_attributes(cfg, 'entrypoint')
  for node in attr:
    if attr[node] == True:
      for index, n in enumerate(cfg.nodes()):
        if n == node:
          entrypoint = index
          break
      break
  return entrypoint

def t_mat(G):
	A = nx.adjacency_matrix(G).todense()
	T = A/A.sum(axis=1)
	return T

def randwalk(G, length, n):
	randwalks = {}
	entrypoint = get_entrypoint(G)
	T = t_mat(G)
	p = T[entrypoint].reshape(-1,1)
	for s in range(n):
		rand_walk = []
		if entrypoint is not None:
			h_a = list(enumerate(G.nodes()))[entrypoint][1] # put entrypoint
			rand_walk.append(h_a)
		for k in range(length):
		    # choose the node with higher probability as the visited node
		    # there may be more than one node with same higher probability,
		    # so we randomly choose the node among this set of high prob.nodes
		    higher_nodes = list()
		    it = np.ndenumerate(p)
		    for idx, x in it:
		    	if not higher_nodes: # first node to be added
		    		if x>0:
		    			higher_nodes.append(idx[1])
		    	elif x>np.max(higher_nodes): # probability of jump > than all the previous nodes
		    		higher_nodes = list()
		    		higher_nodes.append(idx[1])
		    	elif x == np.max(higher_nodes): # equal probability, add it and keep the previous nodes
		    		higher_nodes.append(idx[1])
		    if not higher_nodes: # it's a sink, end of graph reached? then, leave
		    	break
		    else: # else, keep going and update randwalk list
		    	v = random.choice(higher_nodes)
		    	p = T[v]
		    	hex_addr = list(enumerate(G.nodes()))[v][1]
		    	rand_walk.append(hex_addr)
		randwalks[s] = rand_walk
	return randwalks

#define constants for the random walks

LENGTH = 100
N = 50

In [ ]:
# now for each function we retrieve its random walks
# as lists of hex addresses corresponding to the start
# address of each block encountered during the walk

randwalks = {}
for func in func_dict:
  G = json_graph.node_link_graph(func_dict[func])
  randwalks[func] = randwalk(G, LENGTH, N)

print(len(randwalks))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value 

20467


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python3.6

In [ ]:
# save the results
import json
json.dump(randwalks, open("/content/gdrive/My Drive/randwalks.json", "w"))

# further processing has then been carried out to 
# replace each node with its assembly instructions